In [128]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [129]:
# Reproducible results
random_state = 42
np.random.seed(random_state)

# Data

In [130]:
def get_data(one_hot_enc=True, reshape=False, train_val_test=False):
    file_list = os.listdir(DATA_PATH)
    images_path = file_list[0]
    labels_path = file_list[1]
    images_path_full = os.path.join(DATA_PATH + images_path)
    labels_path_full = os.path.join(DATA_PATH + labels_path)
    X = pd.read_csv(images_path_full)
    if reshape:
        X = X.values.astype('float32')
        # Normalize data
        X = X / 255
        # Reshape for cnn 
        X = X.reshape([-1, 28, 28, 1]).astype('float32')
    else:
        # Normalize data
        X = X / 255
    y = pd.read_csv(labels_path_full)
    # Encode labels
    if one_hot_enc:
        encoder = OneHotEncoder(sparse=False, categories='auto')
        y = encoder.fit_transform(y)
    # Divide into train and test set 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    if train_val_test:
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=random_state)
        return X_train, X_val, X_test, y_train, y_val, y_test
    else:
        return X_train, X_test, y_train, y_test

In [178]:
def print_data_shapes_val(X_train, X_val, X_test, y_train, y_val, y_test):
    print("X_train_nn: ", X_train.shape)
    print("X_val_nn: ", X_val.shape)
    print("X_test_nn: ", X_test.shape)
    print("y_train_nn: ", y_train.shape)
    print("y_val_nn: ", y_val.shape)
    print("y_test_nn: ", y_test.shape)

In [179]:
def print_data_shapes_nval(X_train, X_test, y_train, y_test):
    print("X_train_nn: ", X_train.shape)
    print("X_test_nn: ", X_test.shape)
    print("y_train_nn: ", y_train.shape)
    print("y_test_nn: ", y_test.shape)

# Classifiers

### Feed Forward Neural Net

In [131]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras import regularizers

In [140]:
X_train_nn, X_val_nn, X_test_nn, y_train_nn, y_val_nn, y_test_nn = get_data(train_val_test=True)

In [144]:
# Params

# Model compile
adam = Adam(lr=1e-3)
metrics = ['accuracy']
loss = 'categorical_crossentropy'

# Model architecture
regularizer_l2 = regularizers.l2(0.01)

# Model fit
epochs = 12
batch_size = 64

In [180]:
print_data_shapes_val(X_train_nn, X_val_nn, X_test_nn, y_train_nn, y_val_nn, y_test_nn)

X_train_nn:  (39199, 784)
X_val_nn:  (16800, 784)
X_test_nn:  (14000, 784)
y_train_nn:  (39199, 10)
y_val_nn:  (16800, 10)
y_test_nn:  (14000, 10)


In [146]:
model = Sequential()
model.add(Dense(256, input_dim=784, activation='relu', kernel_regularizer=regularizer_l2))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu', kernel_regularizer=regularizer_l2))
model.add(Dense(10, activation='softmax'))

In [147]:
model.compile(loss=loss, optimizer=adam, metrics=metrics)

In [148]:
model.fit(X_train_nn, y_train_nn, validation_data=(X_val_nn, y_val_nn), epochs=epochs, batch_size=batch_size)
scores = model.evaluate(X_test_nn, y_test_nn)

Train on 39199 samples, validate on 16800 samples
Epoch 1/12
39199/39199 [==============================] - 7s 171us/step - loss: 1.0897 - acc: 0.7616 - val_loss: 0.5245 - val_acc: 0.9099
Epoch 2/12
39199/39199 [==============================] - 6s 149us/step - loss: 0.4286 - acc: 0.9320 - val_loss: 0.4009 - val_acc: 0.9348
Epoch 3/12
39199/39199 [==============================] - 7s 191us/step - loss: 0.3719 - acc: 0.9433 - val_loss: 0.3558 - val_acc: 0.9484
Epoch 4/12
39199/39199 [==============================] - 8s 197us/step - loss: 0.3382 - acc: 0.9499 - val_loss: 0.3548 - val_acc: 0.9448
Epoch 5/12
39199/39199 [==============================] - 8s 194us/step - loss: 0.3137 - acc: 0.9541 - val_loss: 0.3261 - val_acc: 0.9509
Epoch 6/12
39199/39199 [==============================] - 8s 192us/step - loss: 0.2997 - acc: 0.9551 - val_loss: 0.3045 - val_acc: 0.9545
Epoch 7/12
39199/39199 [==============================] - 7s 189us/step - loss: 0.2809 - acc: 0.9578 - val_loss: 0.2781 - 

In [149]:
print(scores)

[0.25609567366327557, 0.9591428571428572]


> Can see that the model is somewhat overfitting - Will try to implement *Regularizer*
<br>
This resulted in a loss in accuracy, but there is a trade off between accuracy and overfitting

### CNN

In [150]:
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.datasets import mnist
from keras.utils import np_utils

In [169]:
X_train_cnn, X_val_cnn, X_test_cnn, y_train_cnn, y_val_cnn, y_test_cnn = get_data(one_hot_enc=False, reshape=True, train_val_test=True)

In [170]:
y_train_cnn = np_utils.to_categorical(y_train_cnn)
y_val_cnn = np_utils.to_categorical(y_val_cnn)
y_test_cnn = np_utils.to_categorical(y_test_cnn)

In [181]:
print_data_shapes_val(X_train_cnn, X_val_cnn, X_test_cnn, y_train_cnn, y_val_cnn, y_test_cnn)

X_train_nn:  (39199, 28, 28, 1)
X_val_nn:  (16800, 28, 28, 1)
X_test_nn:  (14000, 28, 28, 1)
y_train_nn:  (39199, 10)
y_val_nn:  (16800, 10)
y_test_nn:  (14000, 10)


In [172]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [173]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [174]:
model.fit(X_train_cnn, y_train_cnn, validation_data=(X_val_cnn, y_val_cnn), epochs=4, batch_size=200)
scores = model.evaluate(X_test_cnn, y_test_cnn)

Train on 39199 samples, validate on 16800 samples
Epoch 1/4
39199/39199 [==============================] - 37s 955us/step - loss: 0.1811 - acc: 0.9437 - val_loss: 0.0731 - val_acc: 0.9781
Epoch 2/4
39199/39199 [==============================] - 37s 938us/step - loss: 0.0577 - acc: 0.9820 - val_loss: 0.0552 - val_acc: 0.9835
Epoch 3/4
39199/39199 [==============================] - 32s 817us/step - loss: 0.0401 - acc: 0.9869 - val_loss: 0.0548 - val_acc: 0.9842
Epoch 4/4
14000/14000 [==============================] - 6s 398us/step


In [175]:
print(scores)

[0.04667685281179313, 0.9867857142857143]


### Random Forest

In [182]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [183]:
X_train_rf, X_test_rf, y_train_rf, y_test_rf = get_data()

In [184]:
print_data_shapes_nval(X_train_rf, X_test_rf, y_train_rf, y_test_rf)

X_train_nn:  (55999, 784)
X_test_nn:  (14000, 784)
y_train_nn:  (55999, 10)
y_test_nn:  (14000, 10)


In [185]:
random_f_clf = RandomForestClassifier(n_estimators=50,
                                     n_jobs = 2,
                                     random_state=random_state)
random_f_clf.fit(X_train_rf, y_train_rf)
y_pred_rf = random_f_clf.predict(X_test_rf)
print(accuracy_score(y_test_rf, y_pred_rf))

0.8957857142857143


### XGBoost

In [35]:
# If needing to install xgboost (using conda)
# ! conda install py-xgboost

In [186]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [187]:
X_train_xg, X_test_xg, y_train_xg, y_test_xg = get_data(one_hot_enc=False)

In [188]:
print_data_shapes_nval(X_train_xg, X_test_xg, y_train_xg, y_test_xg)

X_train_nn:  (55999, 784)
X_test_nn:  (14000, 784)
y_train_nn:  (55999, 1)
y_test_nn:  (14000, 1)


In [190]:
params = {
        'gamma': [0.5, 2],
        'subsample': [0.6, 1.0],
        'colsample_bytree': [0.6, 1.0],
        'max_depth': [3, 5],
        'n_estimators': [5, 10]
        }

Fun to try with more parameters, but my poor CPU cannot take any more..

## Disclaimer: This takes a lot of time

In [191]:
%%capture 
xgb_clf = XGBClassifier()
rs = GridSearchCV(xgb_clf,
                  params,
                  cv=2,
                  scoring="accuracy",
                  n_jobs=1,
                  verbose=2)

rs.fit(X_train_xg, y_train_xg.values.ravel())

In [192]:
optim_est_xg = rs.best_estimator_
print(optim_est_xg)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=0.5, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=10, n_jobs=1, nthread=None, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1.0)


In [193]:
y_pred_xg = optim_est_xg.predict(X_test_xg)
print(accuracy_score(y_test_xg, y_pred_xg))

0.9046428571428572
